In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://bds.com.vn/mua-ban-can-ho-chung-cu-ho-chi-minh-page"
session = requests.Session()

house_data_list = []

# Lặp qua các trang
for page in range(1, 50):  # Bạn có thể thay đổi số trang tại đây
    response = session.get(url + str(page))
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Tìm tất cả các mục bất động sản
    items = soup.find_all('div', class_='item-nhadat bg-white')
    
    for item in items:
        # Lấy tiêu đề và URL của trang chi tiết
        title_tag = item.find('a', class_='title-item-nhadat')
        title = title_tag.get_text(strip=True) if title_tag else ""
        detail_url = title_tag['href'] if title_tag else ""
        
        # Truy cập vào URL chi tiết
        if detail_url:
            full_url = "" + detail_url
            detail_response = session.get(full_url)
            detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
            
            # Lấy thông tin từ trang chi tiết
            
            # Lấy giá, diện tích, ngày đăng, mã tin
            gia_tag = detail_soup.find('span', class_='value-attr')
            gia = gia_tag.get_text(strip=True) if gia_tag else ""
            
            dientich_tag = detail_soup.find('span', text="Diện tích :")
            dientich = dientich_tag.find_next_sibling('span').get_text(strip=True) if dientich_tag else ""

            
            # Lấy số phòng ngủ và số toilet
            so_phong_ngu_tag = detail_soup.find('span', text="Số phòng ngủ :")
            so_phong_ngu = so_phong_ngu_tag.find_next_sibling('span', class_='value-attr2').get_text(strip=True) if so_phong_ngu_tag else ""
            
            so_toilet_tag = detail_soup.find('span', text="Số toilet :")
            so_toilet = so_toilet_tag.find_next_sibling('span', class_='value-attr2').get_text(strip=True) if so_toilet_tag else ""
            
            # Lấy địa chỉ từ breadcrumb
            breadcrumb = detail_soup.find('ul', class_='breadcrumb clearfix')
            loai_bds = ""
            thanh_pho = ""
            huyen = ""
            xa = ""

            if breadcrumb:
                breadcrumb_items = breadcrumb.find_all('li')
                if len(breadcrumb_items) >= 4:
                    loai_bds = breadcrumb_items[0].get_text(strip=True)  # Loại bất động sản
                    thanh_pho = breadcrumb_items[1].get_text(strip=True)  # Thành phố
                    huyen = breadcrumb_items[2].get_text(strip=True)      # Huyện
                    xa = breadcrumb_items[3].get_text(strip=True)         # Xã
            
            # Lưu thông tin vào dictionary
            house_data = {
                'Giá': gia,
                'Diện tích': dientich,
                'Số phòng ngủ': so_phong_ngu,
                'Số toilet': so_toilet,
                'Loại BĐS': loai_bds,
                'Thành phố': thanh_pho,
                'Huyện': huyen,
                'Xã': xa
            }
            
            # Thêm dữ liệu vào danh sách
            house_data_list.append(house_data)

# Tạo DataFrame và lưu vào file Excel
df = pd.DataFrame(house_data_list)
# In ra DataFrame để xem kết quả
print(df)


C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_4860\1437100485.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  dientich_tag = detail_soup.find('span', text="Diện tích :")
C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_4860\1437100485.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  so_phong_ngu_tag = detail_soup.find('span', text="Số phòng ngủ :")
C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_4860\1437100485.py:44: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  so_toilet_tag = detail_soup.find('span', text="Số toilet :")
